In [1]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import numpy as np
import re
import os
maxlen = 256
#这里maxlen = 512会爆内存
epochs = 1
batch_size = 32
learning_rate = 1e-5  # bert_layers越小，学习率应该要越大
crf_lr_multiplier = 1000  # 必要时扩大CRF层的学习率
categories = set()
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'   #指定第一块GPU可用
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.  # 程序最多只能占用指定gpu50%的显存
config.gpu_options.allow_growth = True      #程序按需申请内存
sess = tf.compat.v1.Session(config = config)

In [3]:
config_path = '/home/xiaoguzai/下载/uer/mixed_corpus_bert_large_model/bert_config.json'
checkpoint_path = '/home/xiaoguzai/下载/uer/mixed_corpus_bert_large_model/bert_model.ckpt'
dict_path = '/home/xiaoguzai/下载/uer/mixed_corpus_bert_large_model/vocab.txt'

In [4]:
categories_to_id = {'redundant':0,'prov':1,'city':2,'district':3,'devzone':4,'town':5,'community':6,'village_group':7,'road':8,'roadno':9,'poi':10,'subpoi':11,'houseno':12,'cellno':13,'floorno':14,'roomno':15,'detail':16,'assist':17,'distance':18,'intersection':19,'others':20}
id_to_categories = {categories_to_id[data]:data for data in categories_to_id}

In [5]:
def load_data(filename):
    """加载数据
    单条格式：[text, (start, end, label), (start, end, label), ...]，
              意味着text[start:end + 1]是类型为label的实体。
    """
    data_char = []
    data_id = []
    data_flag = []
    with open(filename, encoding='utf-8') as f:
        f = f.read()
        for l in f.split('\n\n'):
            if not l:
                continue
            current_char = []
            current_id = []
            current_flag = []
            for i, c in enumerate(l.split('\n')):
                try:
                    char, flag = c.split(' ')
                except:
                    print('l.split = ')
                    print(l.split('\n'))
                    print('error c = ')
                    print(c)
                    continue
                flag = flag.split('-')
                if len(flag) == 2:
                    flag1 = flag[0]
                    flag2 = flag[1]
                else:
                    flag1 = flag[0]
                current_char.append(char)
                current_flag.append(flag)
                if flag1 == 'O' or len(flag) == 1:
                    current_id.append(categories_to_id['redundant'])
                else:
                    if len(flag) == 2:
                        print('flag = ')
                        print(flag)
                        if flag1 == 'B' or flag1 == 'S':
                            current_id.append(categories_to_id[flag2]*2+1)
                        #'B'和'S'的时候，flag*2+1
                        else:
                        #'I'和'E'的时候，flag*2
                            current_id.append(categories_to_id[flag2]*2)
            data_char.append(current_char)
            data_id.append(current_id)
            data_flag.append(current_flag)
    return data_char,data_id,data_flag

In [6]:
def new_load_data(filename):
    """加载数据
    单条格式：[text, (start, end, label), (start, end, label), ...]，
              意味着text[start:end + 1]是类型为label的实体。
    对cellno数据进行增强
    """
    data_char = []
    data_id = []
    data_flag = []
    with open(filename, encoding='utf-8') as f:
        f = f.read()
        for l in f.split('\n\n'):
            if not l:
                continue
            current_char = []
            current_id = []
            current_flag = []
            judge_flag = False
            judge_subpoi_flag = False
            judge_cellno_flag = False
            for i, c in enumerate(l.split('\n')):
                try:
                    char, flag = c.split(' ')
                except:
                    print(l.split('\n'))
                    continue
                flag = flag.split('-')
                if len(flag) == 2:
                    flag1 = flag[0]
                    flag2 = flag[1]
                else:
                    flag1 = flag[0]
                current_char.append(char)
                current_flag.append(flag)
                if flag1 == 'O' or len(flag) == 1:
                    current_id.append(categories_to_id['redundant'])
                else:
                    if len(flag) == 2:
                        if flag1 == 'B' or flag1 == 'S':
                            current_id.append(categories_to_id[flag2]*2+1)
                        #'B'和'S'的时候，flag*2+1
                        else:
                        #'I'和'E'的时候，flag*2
                            current_id.append(categories_to_id[flag2]*2)
                        if flag2 == 'intersection':
                            judge_flag = True  
                        if flag2 == 'subpoi':
                            judge_subpoi_flag = True
                        if flag2 == 'cellno':
                            judge_cellno_flag = True
            data_char.append(current_char)
            data_id.append(current_id)
            data_flag.append(current_flag)
            if judge_flag == True:
                data_char.append(current_char)
                data_id.append(current_id)
                data_flag.append(current_flag)
            if judge_subpoi_flag == True:
                data_char.append(current_char)
                data_id.append(current_id)
                data_flag.append(current_flag)
                data_char.append(current_char)
                data_id.append(current_id)
                data_flag.append(current_flag)
                data_char.append(current_char)
                data_id.append(current_id)
                data_flag.append(current_flag)
            if judge_cellno_flag == True:
                data_char.append(current_char)
                data_id.append(current_id)
                data_flag.append(current_flag)
                data_char.append(current_char)
                data_id.append(current_id)
                data_flag.append(current_flag)
    return data_char,data_id,data_flag

In [7]:
train_data,train_id,train_flag = new_load_data('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/train.conll')
evaluate_data,evaluate_id,evaluate_flag = load_data('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/dev.conll')

['浙 B-prov', '江 I-prov', '省 E-prov', '杭 B-city', '州 I-city', '市 E-city', '滨 B-district', '江 I-district', '区 E-district', '滨 B-road', '盛 I-road', '路 E-road', '宝 B-poi', '龙 I-poi', '城 E-poi', 'A B-houseno', '座 E-houseno', '']
flag = 
['B', 'city']
flag = 
['E', 'city']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['I', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['E', 'town']
flag = 
['B'

flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['E', 'district']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['E', 'district']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['I', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'subpoi']
flag = 
['E', 'subpoi']
flag = 
['B', 'subpoi']
flag = 
['I', 'subpoi'

flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['I', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'community']
flag = 
['I', 'community']
flag = 
['E', 'community']
flag = 
['B', 'houseno']
flag = 
['I', 'houseno']
flag = 
['E', 'houseno']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['I', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']

flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['I', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'houseno']
flag = 
['I', 'houseno']
flag = 
['E', 'houseno']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
fl

['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'community']
flag = 
['E', 'community']
flag = 
['B', 'devzone']
flag = 
['I', 'devzone']
flag = 
['I', 'devzone']
flag = 
['I', 'devzone']
flag = 
['E', 'devzone']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'subpoi']
flag = 
['E', 'subpoi']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
fla

...

['E', 'town']
flag = 
['B', 'community']
flag = 
['I', 'community']
flag = 
['E', 'community']
flag = 
['B', 'village_group']
flag = 
['E', 'village_group']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'community']
flag = 
['E', 'community']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['E', 'city']
flag = 
['B', 'road']
fla

flag = 
['E', 'assist']
flag = 
['B', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['E', 'subpoi']
flag = 
['B', 'district']
flag = 
['E', 'district']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['E', 'subpoi']
flag = 
['S', 'assist']
flag = 
['B', 'subpoi']
flag = 
['E', 'subpoi']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['E', 'district']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'subpoi']
flag = 
['E', 'subpoi']
flag = 
['B', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['I', 'subpoi']
flag = 
['E', 'subpoi']
flag = 
['B', 'city']
flag = 
['I

['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['E', 'city']
flag = 
['B', 'town']
flag = 
['E', 'town']
flag = 
['B', 'community']
flag = 
['I', 'community']
flag = 
['E', 'community']
flag = 
['B', 'prov']
flag = 
['I', 'prov']
flag = 
['

flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'prov']
flag = 
['I', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'community']
flag = 
['I', 'community']
flag = 
['E', 'community']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 

['E', 'prov']
flag = 
['B', 'city']
flag = 
['I', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'town']
flag = 
['I', 'town']
flag = 
['I', 'town']
flag = 
['E', 'town']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'roadno']
flag = 
['I', 'roadno']
flag = 
['I', 'roadno']
flag = 
['E', 'roadno']
flag = 
['B', 'prov']
flag = 
['I', 'prov']
flag = 
['E', 'prov']
flag = 
['B', 'city']
flag = 
['E', 'city']
flag = 
['B', 'district']
flag = 
['I', 'district']
flag = 
['E', 'district']
flag = 
['B', 'road']
flag = 
['I', 'road']
flag = 
['I', 'road']
flag = 
['E', 'road']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 
['B', 'town']
flag = 
['E', 'town']
flag = 
['B', 'community']
flag = 
['E', 'community']
flag = 
['B', 'poi']
flag = 
['I', 'poi']
flag = 
['E', 'poi']
flag = 

print(train_data[0:10000])
print(train_id[0:10000])

train_data[0]

In [8]:
# 标注数据
#valid_data,valid_id = load_data('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/dev.conll')
#test_data,test_id = load_data('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/final_test.txt')

In [9]:
from tokenization import FullTokenizer
tokenizer = FullTokenizer(dict_path)
train_token_ids = []
train_label_ids = []
for tokens in train_data:
    tokens = ["[CLS]"]+tokens+["[SEP]"]
    current_token = tokenizer.convert_tokens_to_ids(tokens)
    train_token_ids.append(current_token)
for ids in train_id:
    ids = [0]+ids+[0]
    train_label_ids.append(ids)
evaluate_token_ids = []
evaluate_label_ids = []
for tokens in evaluate_data:
    tokens = ["[CLS]"]+tokens+["[SEP]"]
    current_token = tokenizer.convert_tokens_to_ids(tokens)
    evaluate_token_ids.append(current_token)
for ids in evaluate_id:
    ids = [0]+ids+[0]
    evaluate_label_ids.append(ids)

畈 is not found.
畈 is not found.
畈 is not found.
畈 is not found.
甑 is not found.
塍 is not found.
垅 is not found.
畈 is not found.
畈 is not found.
畈 is not found.
畈 is not found.
樨 is not found.
樨 is not found.
樨 is not found.
樨 is not found.
埭 is not found.
畈 is not found.
塍 is not found.
埭 is not found.
镆 is not found.
塍 is not found.
汊 is not found.
畈 is not found.
埭 is not found.
埭 is not found.
埭 is not found.
埭 is not found.
塍 is not found.
畈 is not found.
埭 is not found.
埭 is not found.
埭 is not found.
鸬 is not found.
塍 is not found.
塍 is not found.
泺 is not found.
泺 is not found.
泺 is not found.
泺 is not found.
莳 is not found.
圹 is not found.
埭 is not found.
埭 is not found.
埭 is not found.
埭 is not found.
埭 is not found.
闳 is not found.
郾 is not found.
郾 is not found.
郾 is not found.
郾 is not found.
笕 is not found.
矸 is not found.
埭 is not found.
埭 is not found.
埭 is not found.
莜 is not found.
畈 is not found.
埭 is not found.
邗 is not found.
塍 is not found.
塍 is not found.
笕 is not

In [10]:
import json
with open(config_path,'r') as load_f:
    config = json.load(load_f)
print('config = ')
print(config)
config['embedding_size'] = config['hidden_size']
config['num_layers'] = config['num_hidden_layers']

config = 
{'attention_probs_dropout_prob': 0.1, 'directionality': 'bidi', 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 1024, 'initializer_range': 0.02, 'intermediate_size': 4096, 'max_position_embeddings': 512, 'num_attention_heads': 16, 'num_hidden_layers': 24, 'pooler_fc_size': 768, 'pooler_num_attention_heads': 12, 'pooler_num_fc_layers': 3, 'pooler_size_per_head': 128, 'pooler_type': 'first_token_transform', 'type_vocab_size': 2, 'vocab_size': 21128}


In [11]:
from models import Bert
from models import ConditionalRandomField
input_ids = keras.layers.Input(shape=(None,),dtype='int32',name='input_ids')
bertmodel = Bert(**config)
output = bertmodel(input_ids)
output = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(output)
#上面的得分为0.7几
#output = keras.layers.LSTM(128,return_sequences=True)(output)
#上面的得分一次为0.81，一次为0.7几
#output = keras.layers.LSTM(128,return_sequences=True)(output)
r"""
output = keras.layers.RNN(keras.layers.LSTMCell(
        units = 3, # 输出
        return_sequences = True,
        input_shape= (None,None,768), # 输入
   ))(output)
"""
#output = keras.layers.Dropout(0.2)(output)
result_dense = keras.layers.Dense(units = len(categories_to_id)*2,
                           kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02),
                           activation = 'softmax',       
                           name = "final_dense")
output = result_dense(output)
#CRF = ConditionalRandomField(lr_multiplier=crf_lr_multiplier)
#output = CRF(output)
#print('output = ')
#print(output)
model = keras.Model(inputs=input_ids,outputs=output)

with call_context.enter


ResourceExhaustedError: in user code:

    /home/xiaoguzai/代码/天体地址序列标注相关代码保存/models.py:176 call  *
        output_embeddings = encoder_layer(output_embeddings)
    /home/xiaoguzai/代码/天体地址序列标注相关代码保存/models.py:435 call  *
        embedding_output = self.dense(embedding_output)
    /home/xiaoguzai/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1038 __call__  **
        self._maybe_build(inputs)
    /home/xiaoguzai/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:2641 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /home/xiaoguzai/.local/lib/python3.8/site-packages/tensorflow/python/keras/layers/core.py:1191 build
        self.kernel = self.add_weight(
    /home/xiaoguzai/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:639 add_weight
        variable = self._add_variable_with_custom_getter(
    /home/xiaoguzai/.local/lib/python3.8/site-packages/tensorflow/python/

In [ ]:
from loader import load_stock_weights
load_stock_weights(bert=bertmodel,ckpt_path=checkpoint_path)

In [ ]:
learning_rate = 2e-5
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    #loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate),
    metrics = [keras.metrics.SparseCategoricalAccuracy(name="acc")]
)
model.summary()

from sklearn.model_selection import StratifiedKFold
seed = 7
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
train_index = [i for i in range(0,len(train_data))]
print('train_index = ')
print(train_index)
for train,test in kfold.split(train_index,train_index):
    print('train = ')
    print(train)

In [ ]:
batch_size = 48
epochs = 1
seed = 7
#batch_size = 48能训练到0.9几，但是得分只有30几
#batch_size = 64能训练到0.9几，但是得分只有30几
#batch_size = 32
#batch_size = 48
#large batch_size = 32
#回头试试batch_size = 48或者batch_size = 40
#from tokenization import sequence_padding
def sequence_padding(inputs):
    max_seq_len = max([len(x) for x in inputs])
    token_type_ids = [0]*max_seq_len
    x = []
    for input_ids in inputs:
        input_ids = input_ids+[0]*(max_seq_len-len(input_ids))
        x.append(np.array(input_ids))
    return np.array(x)
    
class DataGenerator(object):
    def __init__(self,token_ids,label_ids,batch_size=48):
        self.token_ids = token_ids
        self.label_ids = label_ids
        self.batch_size = batch_size
        self.steps = int(np.floor(len(self.token_ids)/self.batch_size))
        self.totals = len(self.token_ids)
        self.maxlen = maxlen
    
    def __len__(self):
        return int(np.floor(len(self.token_ids)/self.batch_size))
    
    def sample(self, random=False):
        """采样函数，每个样本同时返回一个is_end标记
        """
        indices = list(range(len(self.token_ids)))
        np.random.shuffle(indices)
        for i in indices:
            yield self.token_ids[i],self.label_ids[i]
        
    def __iter__(self,random=False):
        random = False
        batch_data = []
        batch_token_ids = []
        batch_label_ids = []
        currents = 0
        for current_token_ids,current_label_ids in self.sample(random):
        #传入的数据在下面定义train_generator = data_generator(train_data, batch_size)
        #这里如果使用tqdm(self.sample(random))，它就会连续地不断产生红色区域
        #如果不使用tqdm(self.sample(random))，它就会连续以...的形式输出进度
        #因为model.fit()函数之中自带相应的进度条
            batch_token_ids.append(current_token_ids)
            batch_label_ids.append(current_label_ids)
            currents = currents+1
            if len(batch_token_ids) == self.batch_size or currents == self.totals:
                #len(batch_token_ids) == self.batch_size:当前批次结束
                #is_end:所有数据结束(可能不够一个批次)
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_label_ids = sequence_padding(batch_label_ids)
                yield batch_token_ids,batch_label_ids
                r"""
                这里的batch_token_ids和batch_segment_ids外面必须加上np.array
                """
                batch_token_ids,batch_label_ids = [],[]
                #每一个批次结束的时候

    def cycle(self,random=True):
        while True:
            for d in self.__iter__(random):
                yield d

In [ ]:
import conlleval
from conlleval import evaluate_from_file
class Evaluator(keras.callbacks.Callback):
    def __init__(self,evalute_token_ids,evaluate_label_ids,evaluate_flag_ids):
        self.best_score = 0.0
        self.evaluate_token_ids = evaluate_token_ids
        self.evaluate_label_ids = evaluate_label_ids
        self.evaluate_flag_ids = evaluate_flag_ids
    
    def on_epoch_end(self,epoch,logs=None):
        wrong_data = []
        label_id = []
        predict_data = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0,20:0}
        true_data = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0,20:0}
        predict_true_data = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0,20:0}
        current_label = None
        f = open('./judge_result.txt','w')
        prelabel = None
        for i in tqdm(range(len(self.evaluate_token_ids))):
            current_result = model(np.array([np.array(self.evaluate_token_ids[i])]))
            current_result = current_result.numpy()
            current_result = current_result.argmax(axis=-1)
            #f.write(str(i+1)+'\u0001')
            #f.write(test_data[i]+'\u0001')
            #current_result = current_result[0][1:-1]
            current_result = current_result[0][1:-1]
            #print('current_result = ')
            #print(current_result)
            tags = []
            for j in range(len(current_result)):
                if current_result[j] == 0:
                    tags.append('O')
                elif current_result[j]%2 == 0:
                    tags.append('I')
                else:
                    tags.append('B')
            new_tags = []
            for j in range(len(tags)):
                if tags[j] == 'O':
                    new_tags.append('O')
                elif tags[j] == 'B':
                #当前位置为'B'
                    if j+1 < len(tags) and tags[j+1] == 'I':
                    #下一位为‘I'
                        new_tags.append('B')
                    else:
                    #下一位为'O'或'E'
                        new_tags.append('S')
                elif tags[j] == 'I':
                #当前位置为'I'
                    if j+1 < len(tags) and tags[j+1] == 'I':
                    #下一位为'I'
                        new_tags.append('I')
                    else:
                    #下一位为'O'或'E'
                        new_tags.append('E')
                    
            for j in range(len(current_result)):
                f.write(tokenizer.inv_vocab[self.evaluate_token_ids[i][j+1]])
                #!!!这里使用self.evaluate_token_ids[i][j+1]的原因:
                #self.evaluate_token_ids发生了错位，而self.evaluate_flag_ids并
                #没有发生错位
                f.write(' '+self.evaluate_flag_ids[i][j][0])
                if len(self.evaluate_flag_ids[i][j]) == 2:
                    f.write('-'+self.evaluate_flag_ids[i][j][1]+' ')
                else:
                    f.write(' ')
                if new_tags[j] == 0:
                #if tag == 'O'
                    f.write('O')
                else:
                    f.write(new_tags[j]+'-')
                    f.write(id_to_categories[current_result[j]//2])
                    f.write('\n')
            f.write('\n')
        f.close()

        #fp = open('./wrong_data.txt','w')
        r"""
        for i in tqdm(range(len(self.evaluate_token_ids))):
            current_result = model(np.array([np.array(self.evaluate_token_ids[i])]))
            current_result = current_result.numpy()
            current_result = current_result.argmax(axis=-1)
            origin_result = current_result
            current_id = current_result//2
            current_data = current_id[0]
            #进行预测
            #fp.write('i = %d\n'%i)
            for j in range(len(current_data)):
                #打上标签
                current_label = self.evaluate_label_ids[i][j]//2
                predict_data[current_data[j]] = predict_data[current_data[j]]+1
                true_data[current_label] = true_data[current_label]+1
                if current_data[j] == current_label:
                    #fp.write(str(tokenizer.inv_vocab[self.evaluate_token_ids[i][j]])+'-'+str(id_to_categories[current_data[j]])+'\n')
                    predict_true_data[current_data[j]] = predict_true_data[current_data[j]]+1
                #else:
                    #current_dict = {"wrong_word":tokenizer.inv_vocab[self.evaluate_token_ids[i][j]],"predict_data":id_to_categories[current_data[j]],"true_data":id_to_categories[current_label]}
                    #fp.write(str(current_dict)+'\n')
        #fp.write('\n\n\n\n\n\n')
        #fp.close()
        """
        prec,rec,f1 = evaluate_from_file('judge_result.txt')
        r"""
        score1,score2,score3 = 0,0,0
        total_score1 = {}
        total_score2 = {}
        total_score3 = {}
        total_score = 0
        for data in predict_true_data:
            if data == 0:
                continue
            
            if predict_data[data] == 0:
                score1 = 0
            else:
                score1 = predict_true_data[data]/predict_data[data]
            
            if true_data[data] == 0:
                score2 = 0
            else:
                score2 = predict_true_data[data]/true_data[data]
            
            if score1+score2 == 0:
                score3 = 0
            else:
                score3 = 2*score1*score2/(score1+score2)
            
            total_score1[data] = score1
            total_score2[data] = score2
            total_score3[data] = score3
            total_score = total_score+score1+score2+score3
        print('准确率 = ')
        print(total_score1)
        print('召回率 = ')
        print(total_score2)
        print('f1_score = ')
        print(total_score3)
        print('总分数为')
        print(total_score)
        """
        final_score = (prec+rec+f1)/3
        print('current_score = ')
        print(final_score)
        if final_score > self.best_score:
            self.best_score = final_score
            model.save_weights('./词性标注best_model6.19.h5')
            model.save('bert+softmax+bilstm+dense_the_best_model.h5')
            #model.save存储模型结构优化器等全部内容
            #重新加载的时候使用model.load_model('**.h5')进行加载

In [ ]:
data_generator = DataGenerator(train_token_ids,train_label_ids,batch_size)
evaluator = Evaluator(evaluate_token_ids,evaluate_label_ids,evaluate_flag)

evaluator.on_epoch_end(epoch=1)

In [ ]:
input_ids = keras.layers.Input(shape=(None,),dtype='int32',name='input_ids')
output_ids = model(input_ids)
print(output_ids)

model.load_weights('./词性标注best_model6.19.h5')

In [ ]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
model.fit(
    data_generator.cycle(),
    steps_per_epoch = len(data_generator),
    epochs = 10,
    callbacks = [evaluator]
)

In [ ]:
model.load_weights('./词性标注best_model6.19.h5')

model.load_weights('./词性标注best_model.h5')

In [ ]:
f = open("/home/xiaoguzai/数据/天池比赛地址实体识别数据集/final_test.txt")               # 返回一个文件对象   
line = f.readline()               # 调用文件的 readline()方法   
test_data = []
import re
while line:
    line = re.sub(r'^([0-9]+)','',line)
    line = re.sub(r'\n','',line)
    test_data.append(line[1:])
    line = f.readline()   
f.close()

In [ ]:
test_str_ids = []
test_label_ids = []
from tokenization import FullTokenizer
tokenizer = FullTokenizer(dict_path)
for data in tqdm(test_data):
    test_str_ids.append(list(data))
    tokens = ["[CLS]"]+list(data)+["[SEP]"]
    current_token = tokenizer.convert_tokens_to_ids(tokens)
    result_label = model(np.array([np.array(current_token)]))
    result_label = np.array(result_label).argmax(axis=-1)
    result_label = result_label[0][1:-1]
    test_label_ids.append(list(result_label))

In [ ]:
len(test_data)

def postdeal(label):
    #categories_to_id = {'redundant':0,'prov':1,'city':2,'district':3,'devzone':4,'town':5,'community':6,'village_group':7,'road':8,'roadno':9,'poi':10,'subpoi':11,'houseno':12,'cellno':13,'floorno':14,'roomno':15,'detail':16,'assist':17,'distance':18,'intersection':19,'others':20}
    for index1 in range(len(label)):
        currentdata = label[index1]
        for index2 in range(len(currentdata)):
            if currentdata[index2]//2 != 17 and index2 > 0 and index2 < len(currentdata)-1 and currentdata[index2-1]//2 == currentdata[index2+1]//2:
            #左右两边标签相同的时候
                if currentdata[index2-1] == 0:
                #为reduntant的时候
                    label[index1][index2] = 0
                elif currentdata[index2-1]%2 == 1 and currentdata[index2+1]%2 == 1:
                #都为终结不动，因为自身可能为终结
                    continue
                elif 
                #其他情况(左不终结右终结，左右都为不终结)时标记为中间的符号
                    label[index1][index2] = currentdata[index2-1]//2*2
            if index2 > 1 and currentdata[index2] == 10 and currentdata[index2-1]//2 != 9 and currentdata[index2-2]//2 != 9:
                label[index1][index2] = 9*2+1
    return label

In [ ]:
f = open('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/results_new6.19.txt','w')
prelabel = None

for i in range(len(test_data)):
    f.write(str(i+1)+'\u0001')
    f.write(test_data[i]+'\u0001')
    tags = []
    for j in range(len(test_label_ids[i])):
        if test_label_ids[i][j] == 0:
            tags.append('O')
        elif test_label_ids[i][j]%2 == 0:
            tags.append('I')
        else:
            tags.append('B')
    new_tags = []
    for j in range(len(tags)):
        if tags[j] == 'O':
            new_tags.append('O')
        elif tags[j] == 'B':
        #当前位置为'B'
            if j+1 < len(tags) and tags[j+1] == 'I':
            #下一位为‘I'
                new_tags.append('B')
            else:
            #下一位为'O'或'E'
                new_tags.append('S')
        elif tags[j] == 'I':
        #当前位置为'I'
            if j+1 < len(tags) and tags[j+1] == 'I':
            #下一位为'I'
                new_tags.append('I')
            else:
            #下一位为'O'或'E'
                new_tags.append('E')
        
    for j in range(len(new_tags)):
        if new_tags[j] == 'O':
            f.write('O')
        else:
            f.write(new_tags[j]+'-')
            f.write(id_to_categories[test_label_ids[i][j]//2])
        if j != len(test_label_ids[i])-1:
            f.write(' ')
        else:
            f.write('\n')

len(test_label_ids)

f = open('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/预测标签6.19.txt','w')
for i in range(len(test_data)):
    f.write(str(i+1)+'\n')
    for j in range(len(test_label_ids[i])):
        f.write(test_data[i][j]+' '+str(test_label_ids[i][j]))
        f.write('\n')
    f.write('\n')

f = open('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/results_new6.19.txt','w')
prelabel = None

for i in range(len(test_data)):
    f.write(str(i+1)+'\u0001')
    f.write(test_data[i]+'\u0001')
    for j in range(len(test_label_ids[i])):
        if test_label_ids[i][j] == 0:
            f.write('O')
            prelabel = 'O'
        else:
            if test_label_ids[i][j]%2 == 0:
                f.write('I-')
                prelabel = 'I'
            else:
                if (j-1 >= 0 and j+1 < len(test_label_ids[i]) and test_label_ids[i][j-1]//2 != test_label_ids[i][j]//2
                    and test_label_ids[i][j+1]//2 != test_label_ids[i][j]//2):
                    #f.write('situation1-')
                    f.write('S-')
                elif (j-1 < 0 and j+1 < len(test_label_ids[i]) and test_label_ids[i][j+1]//2 != test_label_ids[i][j]//2):
                    #f.write('situation2-')
                    f.write('S-')
                elif (j-1 >= 0 and j+1 == len(test_label_ids[i]) and test_label_ids[i][j-1]//2 != test_label_ids[i][j]//2):
                    #f.write('situation3-')
                    f.write('S-')
                #前后标签都不同，或者前面到头后面与它标签不同或者后面到头前面与它标签不同
                elif prelabel == 'O' or j == 0:
                    f.write('B-')
                    prelabel = 'B'
                elif prelabel == 'B':
                    f.write('E-')
                    prelabel = 'E'
                elif prelabel == 'E':
                    f.write('B-')
                    prelabel = 'B'
                elif prelabel == 'I':
                    f.write('E-')
                    prelabel = 'E'
            f.write(id_to_categories[test_label_ids[i][j]//2])
        if j != len(test_label_ids[i])-1:
            f.write(' ')
        else:
            f.write('\n')

f = open('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/analyzeresults.txt','w')
prelabel = None
for i in range(len(test_data)):
    f.write(str(i+1)+'\u0001')
    f.write(test_data[i]+'\u0001')
    for j in range(len(test_label_ids[i])):
        f.write(test_data[i][j]+':')
        if test_label_ids[i][j] == 0:
            f.write('O')
            prelabel = 'O'
        else:
            if test_label_ids[i][j]%2 == 0:
                f.write('I-')
                prelabel = 'I'
            else:
                if (j-1 >= 0 and j+1 < len(test_label_ids[i]) and test_label_ids[i][j-1]//2 != test_label_ids[i][j]//2
                    and test_label_ids[i][j+1]//2 != test_label_ids[i][j]//2):
                    #f.write('situation1-')
                    f.write('S-')
                elif (j-1 < 0 and j+1 < len(test_label_ids[i]) and test_label_ids[i][j+1]//2 != test_label_ids[i][j]//2):
                    #f.write('situation2-')
                    f.write('S-')
                elif (j-1 >= 0 and j+1 == len(test_label_ids[i]) and test_label_ids[i][j-1]//2 != test_label_ids[i][j]//2):
                    #f.write('situation3-')
                    f.write('S-')
                #前后标签都不同，或者前面到头后面与它标签不同或者后面到头前面与它标签不同
                elif prelabel == 'O' or j == 0:
                    f.write('B-')
                    prelabel = 'B'
                elif prelabel == 'B':
                    f.write('E-')
                    prelabel = 'E'
                elif prelabel == 'E':
                    f.write('B-')
                    prelabel = 'B'
                elif prelabel == 'I':
                    f.write('E-')
                    prelabel = 'E'
            f.write(id_to_categories[test_label_ids[i][j]//2])
        if j != len(test_label_ids[i])-1:
            f.write(' ')
        else:
            f.write('\n')
f.close()

In [ ]:
f = open('/home/xiaoguzai/数据/天池比赛地址实体识别数据集/验证集数据.txt','w')
for data in evaluate_data:
    f.write(''.join(data))
    f.write('\n')
f.close()